In [0]:
import numpy as np
from numpy import genfromtxt
from sklearn import svm,preprocessing
from sklearn.metrics import accuracy_score
import random
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import numpy.linalg as la
import scipy.io as sio
import pickle
from cvxopt import matrix, solvers
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
def preprocess(W):
    u = np.min(W,axis = 0)
    v = np.max(W,axis = 0)
    l = W.shape[0]
    t = W.shape[1]
    for i in range(l):
        for j in range(t):
            W[i,j] = W[i,j] - u[j]
            W[i,j]/=(v[j]-u[j])
    return W

**Code for datasets:**

The next two sections generate the .p and .m files used for training and testing. You do not need to run this again if those files are loaded.

In [0]:
datasets = []

# Ringnorm
data = []
file = open('/content/ringnorm.data', 'r')
for line in file.readlines():
  data.append([float(x) for x in line.split()])
data = np.array(data)
data = preprocess(data)
print(data)
ringnorm_data = data
datasets.append(('ringnorm', data))

# twonorm
data = []
file = open('/content/twonorm.data', 'r')
for line in file.readlines():
  data.append([float(x) for x in line.split()])
data = np.array(data)
data = preprocess(data)
twonorm_data = data
datasets.append(('twonorm', data))

mat_file = dict()
mat_file['ringnorm_data'] = ringnorm_data
mat_file['twonorm_data'] = twonorm_data


(1372, 5)




```
# This is formatted as code
```

Generate biased training sets using Sugiyama technique

In [0]:
trials = 100 # trials per dataset
training_points = 100 # size of training dataset
testing_points = 500 # size of testing dataset

for name, data in datasets:
  trial = 0
  training_sets = []
  testing_sets = []
  F = data.shape[1]
  G = data.shape[0]
 
  inds = range(G)
  while trial < trials:
    print("trial {}".format(trial))
    c = random.randint(0,F-1) #choosing a random feature index for a trial
    indices = np.random.permutation(G) #shuffling the indices 
    test_inds  = []
    i = 0
    # choosing 500 indices for test set using Sugiyama biasing technique
    while len(test_inds) < testing_points:
        u = indices[i]
        x = data[u,c]
        prob = min(1,4*x*x)
        v = np.random.random()
        if v < prob:
            test_inds.append(u)
        i+=1
    test_inds = np.array(test_inds)
    print(i)
    train_inds = np.random.choice(indices[i:], training_points, replace = False)
    train_mask = np.array([(i in train_inds) for i in inds])

    
    trial += 1
    training_sets.append(train_inds)
    testing_sets.append(test_inds)
  train_inds = np.array(training_sets)
  test_inds = np.array(testing_sets)
  mat_file['train_inds_{}'.format(name)] = train_inds
  mat_file['test_inds_{}'.format(name)] = test_inds
sio.savemat('datasets.mat', mat_file)
dsets = open('datasets.p', 'wb')
pickle.dump(mat_file, dsets)



trial 0
258
trial 1
234
trial 2
416
trial 3
263
trial 4
242
trial 5
536
trial 6
250
trial 7
230
trial 8
233
trial 9
245
trial 10
499
trial 11
253
trial 12
226
trial 13
221
trial 14
226
trial 15
556
trial 16
215
trial 17
465
trial 18
244
trial 19
229
trial 20
229
trial 21
229
trial 22
239
trial 23
484
trial 24
247
trial 25
230
trial 26
527
trial 27
455
trial 28
454
trial 29
495
trial 30
521
trial 31
230
trial 32
239
trial 33
531
trial 34
448
trial 35
553
trial 36
431
trial 37
246
trial 38
510
trial 39
560
trial 40
219
trial 41
233
trial 42
226
trial 43
495
trial 44
424
trial 45
220
trial 46
255
trial 47
249
trial 48
219
trial 49
598
trial 50
441
trial 51
235
trial 52
255
trial 53
530
trial 54
533
trial 55
587
trial 56
474
trial 57
228
trial 58
214
trial 59
242
trial 60
240
trial 61
238
trial 62
258
trial 63
462
trial 64
219
trial 65
242
trial 66
241
trial 67
456
trial 68
255
trial 69
447
trial 70
478
trial 71
226
trial 72
428
trial 73
227
trial 74
461
trial 75
240
trial 76
217
trial 77
